In [1]:
#Edgar Moises Hernandez-Gonzalez
#08/07/19-04/11/19
#Clasificar EEG de habla imaginada con CNN y LSTM, con CV=5

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [3]:
datos = pd.read_csv("Submuestreo01.csv", header=None)
x = datos.iloc[:, 0:3072].values
y = datos.iloc[:, 3072].values

num_classes = 6
n_samples = len(y)
print(x.shape)
print(y.shape)
print(n_samples)

(132, 3072)
(132,)
132


In [4]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [5]:
#convertir de 2D a 4D
x = x.reshape((n_samples, 1, 512, 6))
print(x.shape)

(132, 1, 512, 6)


In [6]:
skf = StratifiedKFold(n_splits = 5)

In [7]:
resultados = np.zeros((2, 5)) #accuracy y loss en cada validacion cruzada
i = 0
for iTrain, jTest in skf.split(x, y):
    print("\nIteracion:", i + 1)
    x_train, x_test = x[iTrain], x[jTest]
    y_train, y_test = y[iTrain], y[jTest]
    
    model = models.Sequential()
    model.add(layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,512,6)))
    model.add(layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(layers.TimeDistributed(layers.Dropout(0.5)))
    model.add(layers.TimeDistributed(layers.MaxPooling1D(pool_size=2)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    model.add(layers.LSTM(100))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs = 20, batch_size = 64, validation_data=(x_test, y_test))
    
    resultados[0, i], resultados[1, i] = model.evaluate(x_test, y_test)
    i += 1


Iteracion: 1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 103 samples, validate on 29 samples
Epoch 1/20
103/103 [==============================] - 6s 59ms/sample - loss: 1.8214 - acc: 0.1262 - val_loss: 1.7876 - val_acc: 0.2069
Epoch 2/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.7605 - acc: 0.2330 - val_loss: 1.7825 - val_acc: 0.1724
Epoch 3/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.7534 - acc: 0.2427 - val_loss: 1.7805 - val_acc: 0.2069
Epoch 4/20
103/103 [==============================] - 0s 1ms/sample - loss: 1.6800 - acc: 0.3981 - val_loss: 1.7865 - val_acc: 0.2069
Epoch 5/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.5986 - acc: 0.3301 - val_loss: 1.8009 - val_acc: 0.2414
Epoch 6/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.4694 

106/106 [==============================] - 0s 2ms/sample - loss: 0.9174 - acc: 0.8679 - val_loss: 1.8607 - val_acc: 0.1154
Epoch 15/20
106/106 [==============================] - 0s 2ms/sample - loss: 0.8659 - acc: 0.9057 - val_loss: 1.8785 - val_acc: 0.1538
Epoch 16/20
106/106 [==============================] - 0s 2ms/sample - loss: 0.7988 - acc: 0.9151 - val_loss: 1.9300 - val_acc: 0.1538
Epoch 17/20
106/106 [==============================] - 0s 1ms/sample - loss: 0.7061 - acc: 0.9340 - val_loss: 1.9850 - val_acc: 0.1538
Epoch 18/20
106/106 [==============================] - 0s 2ms/sample - loss: 0.6212 - acc: 0.9434 - val_loss: 2.0463 - val_acc: 0.1154
Epoch 19/20
106/106 [==============================] - 0s 2ms/sample - loss: 0.5666 - acc: 0.9340 - val_loss: 2.0933 - val_acc: 0.1154
Epoch 20/20
26/26 [==============================] - 0s 497us/sample - loss: 2.1033 - acc: 0.0769

Iteracion: 4
Train on 107 samples, validate on 25 samples
Epoch 1/20
107/107 [=========================

In [8]:
print("loss:", resultados[0])
print("accuracy:", resultados[1])
print("mean accuracy:", resultados[1].mean())

loss: [2.3994472  2.13013673 2.10327768 2.08514428 2.02482653]
accuracy: [0.13793103 0.18518518 0.07692308 0.23999999 0.28      ]
mean accuracy: 0.1840078577399254
